In [1]:
import json
import os
import itertools
from typing import Tuple

import pandas as pd
import numpy as np
import talib as ta
from tqdm import tqdm

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
with open("settings.json") as jsonfile:
    settings = json.load(jsonfile)
    
BACKTEST_FOLDER = settings["filepaths"]["backtest_folder"]
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["optimized_ml_file"])
ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["analysis_file"])

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if (future["volumeUsd24h"] > settings["markets"]["min_volume_usd_24h"] and 
                future["name"] not in settings["markets"]["blacklist"]):
            markets.append(future["name"])

In [5]:
optimzed_ml = pd.read_csv(OPTIMIZEDML_FILEPATH)

In [ ]:
analysis_df = pd.DataFrame(columns=["Name", "AvgReturns", "StdDev", "AvgDrawdown", "RetDevRatio", "MinReturns", "MaxReturns",
             "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio", "MedReturns", "MedNegReturns",
             "MedPosReturns", "MedPosNegRetRatio", "TheDfactor"])

pbar = tqdm(markets)
for market in pbar:
    pbar.set_description(market)
    df = ftx.get_historical_market_data(market, interval=settings["analysis"]["interval"], start_time=settings["analysis"]["start_time"])
    
    if len(df) < settings["analysis"]["min_data_length"]:
        continue
        
    multiplier = optimzed_ml.loc[optimzed_ml['Name'] == market]["Multiplier"].values[0]
    lookback = optimzed_ml.loc[optimzed_ml['Name'] == market]["Lookback"].values[0]
    
    profits = bt.backtest_dataframe(df, look_back=lookback, multiplier=multiplier)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

XRP-PERP:  97%|█████████▋| 31/32 [01:36<00:03,  3.05s/it]  

In [ ]:
analysis_df

In [ ]:
#analysis_df = analysis_df.round(2)
#analysis_df.to_csv(ANALYSIS_FILEPATH, index=False)

In [ ]:
#pd.read_csv(ANALYSIS_FILEPATH).sort_values("TheDfactor", ascending=False) 